In [1]:
!pip install soundfile
!pip install numpy
!pip install scipy

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [2]:
import os
import glob
import soundfile as sf
import numpy as np
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=5):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a

def butter_bandpass_filter(data, lowcut, highcut, fs, order=5):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

def pre_emphasis(signal, alpha=0.1):
    pre_emphasized_signal = lfilter([1, -alpha], [1], signal)
    return pre_emphasized_signal

def hamming(window_length):
    window = 0.54 - 0.46 * np.cos(2 * np.pi * np.arange(window_length) / (window_length - 1))
    return window

# Parameters for digitization and endpoint detection
lowcut = 300.0
highcut = 3400.0
fs = 8000
n_bits = 16
high_threshold = 500  # Adjust based on your audio characteristics
low_threshold = 400   # Adjust based on your audio characteristics

# Path to the directory containing folders with .wav files
root_path = "d:\Sistema\Escritorio\Escritorio\Tesis\DAIC-WOZ\data"

# Process each folder and .wav file
for folder_name in os.listdir(root_path):
    folder_path = os.path.join(root_path, folder_name)
    if os.path.isdir(folder_path):
        # Find all .wav files in the current folder
        for wav_file_path in glob.glob(os.path.join(folder_path, '*.wav')):
            # Read the audio signal using soundfile (for efficiency)
            audio_signal, original_fs = sf.read(wav_file_path)

            # Pre-filtering
            filtered_signal = butter_bandpass_filter(audio_signal, lowcut, highcut, original_fs)

            # Sampling (downsampling if necessary)
            if original_fs > fs:
                sampled_signal = filtered_signal[::int(original_fs / fs)]
            else:
                sampled_signal = filtered_signal

            # Pre-emphasis
            pre_emphasized_signal = pre_emphasis(sampled_signal)  # Apply pre-emphasis

            # Initialize variables for endpoint detection
            in_speech_segment = False
            speech_segments = []

            # Window framing with endpoint detection
            window_length = 25  # Example window length (adjust as needed)
            hamming_window = hamming(window_length)
            framed_signal = np.zeros((len(pre_emphasized_signal) // window_length, window_length))  # Pre-allocate array

            for i in range(0, len(pre_emphasized_signal) - window_length + 1, window_length):
                start = i
                end = i + window_length

                # Calculate short-time energy
                windowed_frame = pre_emphasized_signal[start:end] * hamming_window
                energy = np.sum(windowed_frame**2)

                # Endpoint detection logic
                if energy > high_threshold and not in_speech_segment:
                    in_speech_segment = True
                    speech_segment_start = start
                elif in_speech_segment and energy < low_threshold:
                    in_speech_segment = False
                    speech_segments.append([speech_segment_start, end])

                # Framing (only for speech segments)
                if in_speech_segment:
                    framed_signal[i // window_length] = windowed_frame

             # Quantization and coding
            quantized_signal = np.zeros_like(framed_signal, dtype=np.int16)
            for i, (segment_start, segment_end) in enumerate(speech_segments):
                quantized_signal[i] = np.int16(pre_emphasized_signal[segment_start:segment_end] * (2**(n_bits - 1) - 1))

            # Join the processed speech segments
            joined_speech = np.concatenate(quantized_signal)

            # Extract the audio file name without extension
            audio_file_name = os.path.splitext(os.path.basename(wav_file_path))[0]
            print(f'Speech segments in {audio_file_name}: {len(speech_segments)}')

            # Save the joined speech
            final_path = os.path.join(folder_path, f'preprocessed_{audio_file_name}.wav')
            sf.write(final_path, joined_speech, fs)
            


c:\Users\herna\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Speech segments in 300_AUDIO: 0
Speech segments in preprocessed_300_AUDIO: 0
Speech segments in 301_AUDIO: 0
Speech segments in preprocessed_301_AUDIO: 0
Speech segments in 302_AUDIO: 0
Speech segments in preprocessed_302_AUDIO: 0
Speech segments in 303_AUDIO: 0
Speech segments in preprocessed_303_AUDIO: 0
Speech segments in 304_AUDIO: 0
Speech segments in preprocessed_304_AUDIO: 0
Speech segments in 305_AUDIO: 0
Speech segments in preprocessed_305_AUDIO: 0
Speech segments in 306_AUDIO: 0
Speech segments in preprocessed_306_AUDIO: 0
Speech segments in 307_AUDIO: 0
Speech segments in preprocessed_307_AUDIO: 0
Speech segments in 308_AUDIO: 0
Speech segments in preprocessed_308_AUDIO: 0
Speech segments in 309_AUDIO: 0
Speech segments in preprocessed_309_AUDIO: 0
Speech segments in 310_AUDIO: 0
Speech segments in preprocessed_310_AUDIO: 0
Speech segments in 311_AUDIO: 0
Speech segments in preprocessed_311_AUDIO: 0
Speech segments in 312_AUDIO: 0
Speech segments in preprocessed_312_AUDIO: 0

KeyboardInterrupt: 